In [1]:
import torch as t
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer, PreTrainedTokenizer, PreTrainedModel
t.cuda.empty_cache()

import sys
sys.path.append('..')  # Add parent directory to path
from hf import HF

In [3]:
model_name = "Qwen/Qwen2.5-14B-Instruct"

In [4]:
base_model, tokenizer = HF.load_model(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# Get the embedding layer
embedding_layer = base_model.get_input_embeddings()  # Embedding matrix
embedding_weights = embedding_layer.weight  # Actual embeddings

In [74]:
def loop_body(text: str, tokenizer: PreTrainedTokenizer, base_model: PreTrainedModel, embedding_layer: t.nn.Module):
  # Tokenize
  tokenized_input = tokenizer(text, return_tensors="pt")
  input_ids = tokenized_input["input_ids"]
  
  # Get embeddings
  input_embeddings = embedding_layer(input_ids)
  
  # Pass through model
  output = base_model(
    inputs_embeds=input_embeddings,
    attention_mask=tokenized_input["attention_mask"]
  )
  
  # Get next token prediction from last position
  next_token_logits = output.logits[0, -1, :]
  next_token_id = t.argmax(next_token_logits).item()
  next_token = tokenizer.decode([next_token_id])
  
  # Return the predicted token
  return next_token

In [75]:
# Get the embedding layer
embedding_layer = base_model.get_input_embeddings()
embedding_weights = embedding_layer.weight

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant



In [95]:
input_text = "Hello, how are you?"

# Create conversation with current assistant text
conversation = [
    {"role": "user", "content": input_text},
]

# Apply chat template with current assistant text
text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

print(text)

# Define end tokens to check for
end_tokens = [tokenizer.eos_token, tokenizer.pad_token]
finished = False

# Simple generation loop
while not finished:
  next_token = loop_body(
    text=text,
    base_model=base_model,
    tokenizer=tokenizer,
    embedding_layer=embedding_layer
  )
  print(next_token, end="", flush=True)
  text += next_token
  
  # Check if any end token is in the most recent output
  for end_token in end_tokens:
    if end_token and end_token in text[-len(end_token)*2:]:
      finished = True
      break
  
  # Safety check for max length
  if len(text) > 4000:
    print("\n[Maximum length reached]")
    finished = True

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant

Hello

! I'm an AI and don't have feelings, but I'm here and ready to help you. How can I assist you today?<|im_end|>